# Overview
The goal is to make a nice retinopathy model by using a pretrained inception v3 as a base and retraining some modified final layers with attention

This can be massively improved with 
* high-resolution images
* better data sampling
* ensuring there is no leaking between training and validation sets, ```sample(replace = True)``` is real dangerous
* better target variable (age) normalization
* pretrained models
* attention/related techniques to focus on areas

## Load Model

In [ ]:
import numpy as np
import pandas as pd
from keras.applications import inception_v3 as inc_net
# Display
from IPython.display import Image
import matplotlib.pyplot as plt
import sklearn
from skimage.segmentation import mark_boundaries
import lime
from lime import lime_image
from keras.preprocessing import image
import os
import shap
import keras.backend as K
from keras.applications.vgg16 import preprocess_input

In [ ]:
import keras
from keras.models import load_model
from keras.metrics import top_k_categorical_accuracy
def top_2_accuracy(in_gt, in_pred):
    return top_k_categorical_accuracy(in_gt, in_pred, k=2)
model = load_model('../input/model-full/full_retina_model.h5',custom_objects={'top_2_accuracy': top_2_accuracy})


In [ ]:
model.load_weights('../input/weights/retina_weights.best.hdf5')

In [ ]:
base_image_dir = os.path.join('..', 'input', 'diabetic-retinopathy-detection')
retina_df = pd.read_csv(os.path.join(base_image_dir, 'trainLabels.csv'))
retina_df['PatientId'] = retina_df['image'].map(lambda x: x.split('_')[0])
retina_df['path'] = retina_df['image'].map(lambda x: os.path.join(base_image_dir,
                                                         '{}.jpeg'.format(x)))
retina_df['exists'] = retina_df['path'].map(os.path.exists)

In [ ]:
def transform_img_fn(path_list):
    '''Transform image so it can be processed by inception.'''
    out = []
    for img_path in path_list:
        img = image.load_img(img_path, target_size=(512, 512))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = inc_net.preprocess_input(x)
        out.append(x)
    return np.vstack(out)

In [ ]:
def retorna_data(classe):
    dataframe = retina_df[retina_df['level'] == classe]
    dataframe = dataframe[dataframe['exists'] == True]
    dataframe['new_index'] = range(len(dataframe))
    dataframe = dataframe.set_index('new_index')
    colunas = ['NO-DR','NPDR-LIGHT','NPDR-MODERATE','NPDR-SEVERE','PDR']
    y = list()
    for i in range(len(dataframe)):
        expl = transform_img_fn([dataframe['path'][i]])
        result = model.predict(expl,batch_size = 32, verbose = True)
        x = [result[0][0],result[0][1],result[0][2],result[0][3],result[0][4]]
        y.append(x)
    return pd.DataFrame(y, columns= colunas)

### Case example data NO-DR

In [ ]:
retorna_data(0)

### Case NPDR-LIGHT

In [ ]:
retorna_data(1)

### Case NPDR-MODERATE

In [ ]:
retorna_data(2)

### Case NPDR-SEVERE	

In [ ]:
retorna_data(3)

### Case PDR

In [ ]:
retorna_data(4)

### Testes explicação

In [ ]:
amostra = np.ndarray(shape=(50, 512, 512,3),
                     dtype=np.float32)
count = 0
for i in range(50):
    to_explain = retina_df['path'][1]
    to_explain = transform_img_fn([to_explain])
    a = model.predict(to_explain,batch_size = 32)
    for j in range(4):
        if((a[0][j] * 100) > 85):
            print(a[0][j])
            amostra[count] = to_explain
            count += 1
    

In [ ]:
def transform_img_fn2(path_list):
    '''Transform image so it can be processed by inception.'''
    out = []
    for img_path in path_list:
        img = image.load_img(img_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = inc_net.preprocess_input(x)
        out.append(x)
    return np.vstack(out)

## DataSet

In [ ]:
to_explain = retina_df['path'][1]
to_explain = transform_img_fn([to_explain])

In [ ]:
to_explain2 = retina_df['path'][2]
to_explain2 = transform_img_fn([to_explain2])

In [ ]:
test = np.ndarray(shape=(2, 512, 512,3),
                     dtype=np.float32)
test[0] = to_explain
test[1] = to_explain2


In [ ]:
amostra = np.ndarray(shape=(5, 512, 512,3),
                     dtype=np.float32)
z = 0
for i in range(5):
    x = retina_df['path'][i]
    x = transform_img_fn([x])
    amostra[z] = x
    z += 1

In [ ]:
test.shape

In [ ]:
to_explain.shape

In [ ]:
def map2layer(x, layer):
    print(x.shape)
    feed_dict = dict(zip([model.layers[0].input], [preprocess_input(x.copy())]))
    #print(feed_dict)
    return K.get_session().run(model.layers[layer].input, feed_dict)

In [ ]:
e = shap.DeepExplainer(model,amostra)

In [ ]:
e = shap.GradientExplainer(
    (model.layers[7].input, model.layers[-1].output),
    map2layer(amostra, 7),
    local_smoothing=0 # std dev of smoothing noise
)

In [ ]:
z = map2layer(amostra, 7)

In [ ]:
z.shape

In [ ]:
x = map2layer(test, 7)

In [ ]:
K.clear_session()

In [ ]:
shap_values,indexes = e.shap_values(map2layer(test, 7), ranked_outputs=4)

In [ ]:
test = np.ndarray(shape=(2, 512, 512,3),
                     dtype=np.float32)
test[0] = to_explain
test[1] = to_explain2

In [ ]:
shap_values = e.shap_values(to_explain)

In [ ]:
shap.image_plot(shap_values, -to_explain)

In [ ]:
index_names = [0,1,2,3,4]

In [ ]:
shap.image_plot(shap_values, to_explain, index_names)

In [ ]:
from keras.preprocessing import image
img_orig = transform_img_fn([x])

In [ ]:
test = list()

In [ ]:
y = retina_df['path'][7]
y = transform_img_fn([y])

In [ ]:
test.append(y)

In [ ]:
c = next(iter(test))

## Exemplo SHAP

In [ ]:
e = shap.DeepExplainer (model, img_orig)
shap_values = e.shap_values(c [1 : 4])

In [ ]:
shap.image_plot (shap_values, - c [1 : 4])

## Exemplo LIME

In [ ]:
model.predict(img_orig)

In [ ]:
explainer = lime_image.LimeImageExplainer(verbose=True)

In [ ]:
explanation = explainer.explain_instance(img_orig[0], model.predict, top_labels=5, hide_color=0, num_samples=1000)

In [ ]:
explanation.top_labels

In [ ]:
temp, mask = explanation.get_image_and_mask(0, positive_only=True, num_features=15, hide_rest=False)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))